# Parse, Chunk and Load Documents 

The following notebook executes three steps: 
- **Parsing and Chunking**: The first part of the notebook parses and chunks the documents.  This is done by the [PyPDFLoader](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pypdf) of LangChain. More documentation can be found here: [LangChain API](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html). 
- **Embeddings**: For every chunk an embeddings is created. For these an OpenAI Embeddings model is used: [text-embedding-3-small](https://platform.openai.com/docs/models/embeddings). 
- **Load to Database**: The Documents and Chunks are loaded to Neo4j. This is done using the [Python Driver](https://neo4j.com/docs/api/python-driver/current/) that enables querying from a Python script.

In [ ]:
%pip install pypdf langchain_community langchain langchain_openai IPython neo4j

In [ ]:
import pandas as pd
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from neo4j import Query, GraphDatabase, RoutingControl, Result
import ast
from IPython.display import clear_output

## Get Credentials

In [ ]:
env_file = '../ws.env'

In [ ]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

In [ ]:
documents_path = "../documents/"

## Parse and Chunk Documents

In [ ]:
chunk_size = 1000
chunk_overlap = 100

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    is_separator_regex = False,
)

In [ ]:
directory = os.fsencode(documents_path)
chunk_seq_id = 0
chunks_with_metadata = []

for doc in os.listdir(directory):
    doc_name = os.fsdecode(doc)
    if doc_name != '.DS_Store':
        print(f"Parsing: {doc_name}")
        doc_path = documents_path + doc_name
        loader = PyPDFLoader(doc_path)
        pages = loader.load_and_split()
        num_chunks = 0
        for page in pages:
            chunks = text_splitter.split_text(page.page_content)
            for chunk in chunks:
                d = {
                    'file': page.metadata['source'],
                    'page': page.metadata['page'],
                    'chunk_original': chunk,
                    'num_chuncks': len(chunks),
                    'chunk_seq_id': chunk_seq_id
                }
                chunk_seq_id += 1
                num_chunks += 1
                chunks_with_metadata.append(d.copy())
        print(f"chunked {len(pages)} pages in {num_chunks} chunks")

Create a DataFrame of Chunks

In [ ]:
df = pd.DataFrame.from_dict(chunks_with_metadata)

In [ ]:
df

#### Translate Documents

In [ ]:
llm = ChatOpenAI(temperature=0, model=LLM)
llm.model_name

In [ ]:
def translate_text(text):
    messages = [
        ("system", "You are a helpful assistant that translates German to English. Translate the text. Output should always be in English. If the chunk is is fully in English, just copy input to output. Keep the structure as is. "),
        ("human", text),
    ]
    ai_msg = llm.invoke(messages)
    return ai_msg.content

In [ ]:
df.iloc[16]['chunk_original']

In [ ]:
translate_text(df.iloc[16]['chunk_original'])

In [ ]:
docs_to_translate = [
    documents_path + 'IPID_KV_Sonderklasse_bestHEALTH.pdf', 
    documents_path + 'MIFID_Austria.pdf',
    documents_path + 'Travel_Insurance_Credit_Cards.pdf',
    documents_path + 'Travel_Insurance_Terms_2025.pdf'
]

In [ ]:
docs_to_translate

In [ ]:
df["chunk_eng"] = df.apply(
    lambda row: translate_text(row["chunk_original"]) 
                if row["file"] in docs_to_translate 
                else row["chunk_original"],
    axis=1)

In [ ]:
df.head()

## Create embeddings

Load an embedding model

In [ ]:
embeddings_model = OpenAIEmbeddings(
    model = EMBEDDINGS_MODEL,
    openai_api_key = OPENAI_API_KEY
)

Add an embedding for every chunk in the DataFrame

In [ ]:
df['embedding'] = df['chunk_eng'].apply(lambda x: embeddings_model.embed_query(x))

In [ ]:
df.head()

## Create Neo4j Connection

Setup the Python Driver for Neo4j with the loaded credentials

In [ ]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the Connection

In [ ]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

## Load to Database

Create some constraints

In [ ]:
driver.execute_query(
    'CREATE CONSTRAINT unique_document IF NOT EXISTS FOR (d:Document) REQUIRE d.id IS UNIQUE',
    database_=DATABASE,
    routing_=RoutingControl.WRITE
)

In [ ]:
driver.execute_query(
    'CREATE CONSTRAINT unique_chunk IF NOT EXISTS FOR (c:Chunk) REQUIRE c.id IS UNIQUE',
    database_=DATABASE,
    routing_=RoutingControl.WRITE
)

In [ ]:
schema_result_df  = driver.execute_query(
    'SHOW CONSTRAINTS',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)
schema_result_df.head()

### Load Documents Nodes to database

Create Dataframe from the documents

In [ ]:
document_df = df['file'].drop_duplicates().copy()
document_df = document_df.reset_index().drop('index',axis=1).reset_index()
document_df = document_df.rename(columns={"index": "doc_id", "file": "file_location"})
document_df['file_name'] = document_df['file_location'].apply(lambda x: x.split('/')[-1])
document_df

Get number of pages per file

In [ ]:
df = pd.merge(df, document_df, left_on='file', right_on='file_location', how='left').copy()

In [ ]:
df

In [ ]:
pages_df = df.groupby(['doc_id', 'file_name']).max(['page'])['page'].apply(lambda x: x+1)

In [ ]:
document_df = pd.merge(document_df, pages_df, on='doc_id', how='left')
document_df

### Load the Documents

In [ ]:
merge_file_query = """
    MERGE(mergedDocument:Document {id: $doc_id})
    SET mergedDocument.file_location = $file_location,
        mergedDocument.file_name = $file_name,
        mergedDocument.pages = $file_pages
    RETURN mergedDocument
    """

In [ ]:
document_df

In [ ]:
for index, row in document_df.iterrows():
    print(row)
    clear_output(wait=True)
    driver.execute_query(
        merge_file_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        doc_id = row.doc_id,
        file_location = row.file_location,
        file_name = row.file_name,
        file_pages = row.page
    )
    print(f"Loaded {row['file_name']}")
    print("Progress: ", np.round((index+1)/document_df.shape[0]*100,2), "%")

### Load Chunk Nodes to database

Create Dataframe for chunks

In [ ]:
chunks_df = df[['chunk_seq_id', 'num_chuncks', 'page', 'chunk_original', 'chunk_eng', 'embedding']]
chunks_df

In [ ]:
merge_chunck_query = """
    MERGE(mergedChunk:Chunk {id: $chunk_seq_id})
        ON CREATE SET
            mergedChunk.page = $page,
            mergedChunk.chunk_original = $chunk_original,
            mergedChunk.chunk_eng = $chunk_eng,
            mergedChunk.embedding = $embedding
    RETURN mergedChunk
"""

In [ ]:
for index, row in chunks_df.iterrows():
    clear_output(wait=True)
    driver.execute_query(
        merge_chunck_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        chunk_seq_id = row.chunk_seq_id,
        page = row.page,
        chunk_original = row.chunk_original,
        chunk_eng = row.chunk_eng,
        embedding = row.embedding
    )
    print("Progress: ", np.round(((index+1)/chunks_df.shape[0])*100,2), "%")

### Load File to Chunk Relationship

In [ ]:
part_of_df = df[['chunk_seq_id', 'doc_id']].copy()
part_of_df

In [ ]:
merge_part_of_query = """
    MATCH
        (doc:Document {id: $doc_id}),
        (chunk:Chunk {id: $chunk_id})
    MERGE (doc)<-[r:PART_OF]-(chunk)
    RETURN doc.name, type(r), chunk.title
"""

In [ ]:
for index, row in part_of_df.iterrows():
    clear_output(wait=True)
    driver.execute_query(
        merge_part_of_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        doc_id = row.doc_id,
        chunk_id = row.chunk_seq_id
    )
    # print(f"Loaded relationship from document {row['doc_id']} to chunk {row['chunk_seq_id']}")
    print("Progress: ", np.round(((index+1)/part_of_df.shape[0])*100,2), "%")

## Load Chunk to Chunk Relationship

Link the chunks in order by the "NEXT" relationship.

In [ ]:
next_query = """
    MATCH (doc:Document)
    WITH doc
    CALL (doc) {
        MATCH (doc)<-[:PART_OF]-(chunks:Chunk)
        WITH chunks ORDER BY chunks.id ASC
        WITH collect(chunks) as chunk_list
        CALL apoc.nodes.link(
            chunk_list,
            "NEXT",
            {avoidDuplicates: true}
        )
        RETURN size(chunk_list) as size_chunk_list
    }
    WITH doc, size_chunk_list
    RETURN doc, size_chunk_list
"""

In [ ]:
 driver.execute_query(
        next_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

### Create Product Types

Create products associated to the documents

In [ ]:
product_query = """
    MERGE (kranken:ProductType {name: 'Krankenversicherung'})
    MERGE (reise:ProductType {name: 'Reisegepäckversicherung'})
    MERGE (reiseunfall:ProductType {name: 'Reise-Unfallversicherung'})
    MERGE (payment:ProductType {name: 'Consumer Payment Services'})
    MERGE (wertpapierdienst:ProductType {name: 'Wertpapierdienstleistungen'})
    
    
    WITH kranken, reise, reiseunfall, payment, wertpapierdienst
    SET kranken.id = 0
    SET reise.id = 1
    SET reiseunfall.id = 2
    SET payment.id = 4
    SET wertpapierdienst.id = 5
    
    WITH kranken, reise, reiseunfall, payment, wertpapierdienst
    MATCH (doc1:Document {file_name: 'IPID_KV_Sonderklasse_bestHEALTH.pdf'})
    MATCH (doc2:Document {file_name: 'Travel_Insurance_Terms_2025.pdf'})
    MATCH (doc3:Document {file_name: 'Travel_Insurance_Terms_2025.pdf'})
    MATCH (doc4:Document {file_name: 'agb_folder_zadig_rz_en.pdf'})
    MATCH (doc5:Document {file_name: 'MIFID_Austria.pdf'})
  
    MERGE (doc1)-[:RELATED_TO]->(kranken)
    MERGE (doc2)-[:RELATED_TO]->(reise)
    MERGE (doc3)-[:RELATED_TO]->(reiseunfall)
    MERGE (doc4)-[:RELATED_TO]->(payment)
    MERGE (doc5)-[:RELATED_TO]->(wertpapierdienst)
"""

In [ ]:
driver.execute_query(
        product_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

### Create Customers

Create some fake customers in the database

In [ ]:
customer_query = """

WITH 50 as amount, 1950 as startYear, 2021 as endYear, [
   'James', 'John', 'William', 'Thomas', 'George',
    'Henry', 'Charles', 'Edward', 'Robert', 'Richard',
    'Joseph', 'David', 'Daniel', 'Matthew', 'Andrew',
    'Michael', 'Peter', 'Christopher', 'Samuel', 'Benjamin',
    'Jonathan', 'Alexander', 'Nicholas', 'Stephen', 'Paul',
    'Mary', 'Elizabeth', 'Catherine', 'Margaret', 'Anne',
    'Sarah', 'Emily', 'Jane', 'Alice', 'Charlotte',
    'Victoria', 'Emma', 'Grace', 'Hannah', 'Lucy',
    'Rachel', 'Rebecca', 'Laura', 'Julia', 'Caroline',
    'Anna', 'Susan', 'Martha', 'Eleanor', 'Rose'
 ] AS firstNames,
[
    'Smith', 'Johnson', 'Williams', 'Brown', 'Jones',
    'Miller', 'Davis', 'Garcia', 'Rodriguez', 'Martinez',
    'Hernandez', 'Lopez', 'Gonzalez', 'Wilson', 'Anderson',
    'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin',
    'Lee', 'Perez', 'Thompson', 'White', 'Harris',
    'Sanchez', 'Clark', 'Ramirez', 'Lewis', 'Robinson',
    'Walker', 'Young', 'Allen', 'King', 'Scott',
    'Green', 'Baker', 'Adams', 'Nelson', 'Carter',
    'Mitchell', 'Perez', 'Roberts', 'Turner', 'Phillips',
    'Campbell', 'Parker', 'Evans', 'Edwards', 'Collins'
] AS lastNames
UNWIND range(1, amount) AS i
CALL (i, firstNames, lastNames, startYear, endYear){
    WITH i,
       apoc.coll.randomItem(firstNames) AS firstName,
       apoc.coll.randomItem(lastNames) AS lastName,
       date({
          year:toInteger(1+startYear+floor(rand()*(endYear - startYear))), 
          month:toInteger(1+floor(rand()*12)), 
          day:toInteger(1+floor(rand()*27))
        }) as birth_date
  WITH i, firstName, lastName, firstName + ' ' + lastName AS fullName, birth_date
  CREATE (c:Customer {
    id: i,
    firstName: firstName,
    lastName: lastName,
    name: fullName,
    birth_date: birth_date
  })
}
"""

In [ ]:
driver.execute_query(
        customer_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

### Link Customers to Products

Randomly link customers to products

In [ ]:
products_query = """
    MATCH (c:Customer)
    // Randomly assign 1 to 3 product types
    WITH c, [
        {name: 'Krankenversicherung'},
        {name: 'Reisegepäckversicherung'},
        {name: 'Reise-Unfallversicherung'},
        {name: 'Consumer Payment Services'},
        {name: 'Wertpapierdienstleistungen'}
    ] AS productTypes
    WITH c, apoc.coll.shuffle(productTypes)[..toInteger(rand()*3)+1] AS selectedTypes
    UNWIND selectedTypes AS pt
    MATCH (ptNode:ProductType {name: pt.name})
    
    CALL (pt) {
        RETURN 
            CASE pt.name
                WHEN 'Consumer Payment Services' THEN 'AT' + apoc.text.random(2, "0-9") + '20111' + apoc.text.random(11, "0-9")
                ELSE NULL 
            END AS iban,
            randomUUID() AS productId,
            date({
                year: toInteger(1 + 2017 + floor(rand() * (2021 - 2017))),
                month: toInteger(1 + floor(rand() * 12)),
                day: toInteger(1 + floor(rand() * 27))
            }) AS expirationDate
    }

    CREATE (p:Product {
        id: productId,
        iban: iban,
        name: pt.name + ' Product',
        expirationDate: expirationDate
    })
    MERGE (c)-[:HAS_PRODUCT]->(p)
    MERGE (p)-[:OF_TYPE]->(ptNode)
    RETURN count(*) AS created
"""

In [ ]:
driver.execute_query(
        products_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
)